# DGD算法+小值攻击

攻击方式：拜占庭用户在每次迭代中始终保持相同的恶意对偶变量（价格变量）（不进行更新），并将该恶意资源量发送给邻居

In [3]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [4]:
#定义变量 
agent_resource_quantity_attack1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_attack1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_attack1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_attack1 = 0.6
dual_variable_attack1 = np.zeros(shape=(100,60000))#对偶变量
proportion_attack1 = 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
W_attack1 =np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt")
iteration_attack1=[]#绘图横坐标
a_attack1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_attack1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [5]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_attack1*100)):
    dual_variable_attack1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_attack1*100)),100):
    dual_variable_attack1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_attack1*100)):
    agent_resource_quantity_attack1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_attack1=0
for i in range (0,100-int(proportion_attack1*100)):
    sum_attack1 = sum_attack1 + agent_resource_quantity_attack1[i][0]
average_resource_honest_attack1[0][0]=sum_attack1/(100-int(proportion_attack1*100))

#诚实用户平均成本初始化
sum_attack1=0
for i in range(0,100-int(proportion_attack1*100)):
    sum_attack1=sum_attack1+a_attack1[i]*(agent_resource_quantity_attack1[i][0]-b_attack1[i])*(agent_resource_quantity_attack1[i][0]-b_attack1[i])
average_cost_honest_attack1[0][0]=sum_attack1/(100-int(proportion_attack1*100))

iteration_attack1.append(0)

In [6]:
#DGD算法 
def DGD_attack1(a_attack1,b_attack1,agent_resource_quantity_attack1,average_resource_honest_attack1,average_cost_honest_attack1,iteration_update_attack1,dual_variable_attack1,W_attack1,iteration_attack1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_attack1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_attack1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_attack1.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_attack1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_attack1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_attack1.append(dual_variable_attack1[i][k-1]-iteration_update_attack1*((50/100)-(1/100)*agent_resource_quantity_attack1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_attack1*100)),100):
            dual_variable_intermediate_attack1.append(dual_variable_attack1[i][k-1])
        #阶段二：聚合 每个用户聚合自身以及邻居的对偶值   
        for i in range(0,100-int(proportion_attack1*100)):
            mix_sum_dual_attack1=0
            for j in range(0,100):
                if W_attack1[i][j]!=0:
                    mix_sum_dual_attack1=mix_sum_dual_attack1+W_attack1[i][j]*dual_variable_intermediate_attack1[j]
            dual_variable_attack1[i][k]=mix_sum_dual_attack1  
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_attack1*100),100):
            dual_variable_attack1[i][k]=dual_variable_attack1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_attack1*100)):
            agent_resource_quantity_attack1[i][k]=(dual_variable_attack1[i][k]-2*a_attack1[i]*b_attack1[i])/(-2*a_attack1[i])
            #原变量的投影操作
            if agent_resource_quantity_attack1[i][k] < 0:
                agent_resource_quantity_attack1[i][k]=0
            elif agent_resource_quantity_attack1[i][k] > 100: 
                agent_resource_quantity_attack1[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_attack1*100),100):
            agent_resource_quantity_attack1[i][k]=agent_resource_quantity_attack1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_attack1=0
        for i in range (0,100-int(proportion_attack1*100)):
            sum1_attack1 = sum1_attack1 + agent_resource_quantity_attack1[i][k]
        average_resource_honest_attack1[0][k]=sum1_attack1/(100-int(proportion_attack1*100))
        
        #计算所有诚实用户的平均成本
        sum2_attack1=0
        for i in range(0,100-int(proportion_attack1*100)):
            sum2_attack1=sum2_attack1+a_attack1[i]*(agent_resource_quantity_attack1[i][k]-b_attack1[i])*(agent_resource_quantity_attack1[i][k]-b_attack1[i])
        average_cost_honest_attack1[0][k]=sum2_attack1/(100-int(proportion_attack1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_attack1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_attack1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_attack1)
            print("****************************************************************************")
        
        iteration_update_attack1=1/(pow(k+1,0.2)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [7]:
DGD_attack1(a_attack1,b_attack1,agent_resource_quantity_attack1,average_resource_honest_attack1,average_cost_honest_attack1,iteration_update_attack1,dual_variable_attack1,W_attack1,iteration_attack1)

****************************DGD算法+小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 100.0
agent 2 在 3999 次迭代中的资源申请量为: 100.0
agent 3 在 3999 次迭代中的资源申请量为: 100.0
agent 4 在 3999 次迭代中的资源申请量为: 100.0
agent 5 在 3999 次迭代中的资源申请量为: 100.0
agent 6 在 3999 次迭代中的资源申请量为: 100.0
agent 7 在 3999 次迭代中的资源申请量为: 100.0
agent 8 在 3999 次迭代中的资源申请量为: 100.0
agent 9 在 3999 次迭代中的资源申请量为: 100.0
agent 10 在 3999 次迭代中的资源申请量为: 100.0
agent 11 在 3999 次迭代中的资源申请量为: 100.0
agent 12 在 3999 次迭代中的资源申请量为: 100.0
agent 13 在 3999 次迭代中的资源申请量为: 100.0
agent 14 在 3999 次迭代中的资源申请量为: 100.0
agent 15 在 3999 次迭代中的资源申请量为: 100.0
agent 16 在 3999 次迭代中的资源申请量为: 100.0
agent 17 在 3999 次迭代中的资源申请量为: 100.0
agent 18 在 3999 次迭代中的资源申请量为: 100.0
agent 19 在 3999 次迭代中的资源申请量为: 100.0
agent 20 在 3999 次迭代中的资源申请量为: 100.0
agent 21 在 3999 次迭代中的资源申请量为: 100.0
agent 22 在 3999 次迭代中的资源申请量为: 100.0
agent 23 在 3999 次迭代中的资源申请量为: 100.0
agent 24 在 3999 次迭代中的资源申请量为: 100.0
agent 25 在 3999 次迭代中的资源申请量为: 100.0
agent 26 在 3999 次迭代中的资源申请量为: 100.0
agent 27 在 3999 次迭代中的资源申请量为: 100.0
agent 28 在 3999 次迭代中的资源申请量为: 100.0
agent 29 在 3999

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 100.0
agent 2 在 5999 次迭代中的资源申请量为: 100.0
agent 3 在 5999 次迭代中的资源申请量为: 100.0
agent 4 在 5999 次迭代中的资源申请量为: 100.0
agent 5 在 5999 次迭代中的资源申请量为: 100.0
agent 6 在 5999 次迭代中的资源申请量为: 100.0
agent 7 在 5999 次迭代中的资源申请量为: 100.0
agent 8 在 5999 次迭代中的资源申请量为: 100.0
agent 9 在 5999 次迭代中的资源申请量为: 100.0
agent 10 在 5999 次迭代中的资源申请量为: 100.0
agent 11 在 5999 次迭代中的资源申请量为: 100.0
agent 12 在 5999 次迭代中的资源申请量为: 100.0
agent 13 在 5999 次迭代中的资源申请量为: 100.0
agent 14 在 5999 次迭代中的资源申请量为: 100.0
agent 15 在 5999 次迭代中的资源申请量为: 100.0
agent 16 在 5999 次迭代中的资源申请量为: 100.0
agent 17 在 5999 次迭代中的资源申请量为: 100.0
agent 18 在 5999 次迭代中的资源申请量为: 100.0
agent 19 在 5999 次迭代中的资源申请量为: 100.0
agent 20 在 5999 次迭代中的资源申请量为: 100.0
agent 21 在 5999 次迭代中的资源申请量为: 100.0
agent 22 在 5999 次迭代中的资源申请量为: 100.0
agent 23 在 5999 次迭代中的资源申请量为: 100.0
agent 24 在 5999 次迭代中的资源申请量为: 100.0
agent 25 在 5999 次迭代中的资源申请量为: 100.0
agent 26 在 5999 次迭代中的资源申请量为: 100.0
agent 27 在 5999 次迭代中的资源申请量为: 100.0
agent 28 在 5999 次迭代中的资源申请量为: 100.0
agent 29 在 5999

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 100.0
agent 2 在 7999 次迭代中的资源申请量为: 100.0
agent 3 在 7999 次迭代中的资源申请量为: 100.0
agent 4 在 7999 次迭代中的资源申请量为: 100.0
agent 5 在 7999 次迭代中的资源申请量为: 100.0
agent 6 在 7999 次迭代中的资源申请量为: 100.0
agent 7 在 7999 次迭代中的资源申请量为: 100.0
agent 8 在 7999 次迭代中的资源申请量为: 100.0
agent 9 在 7999 次迭代中的资源申请量为: 100.0
agent 10 在 7999 次迭代中的资源申请量为: 100.0
agent 11 在 7999 次迭代中的资源申请量为: 100.0
agent 12 在 7999 次迭代中的资源申请量为: 100.0
agent 13 在 7999 次迭代中的资源申请量为: 100.0
agent 14 在 7999 次迭代中的资源申请量为: 100.0
agent 15 在 7999 次迭代中的资源申请量为: 100.0
agent 16 在 7999 次迭代中的资源申请量为: 100.0
agent 17 在 7999 次迭代中的资源申请量为: 100.0
agent 18 在 7999 次迭代中的资源申请量为: 100.0
agent 19 在 7999 次迭代中的资源申请量为: 100.0
agent 20 在 7999 次迭代中的资源申请量为: 100.0
agent 21 在 7999 次迭代中的资源申请量为: 100.0
agent 22 在 7999 次迭代中的资源申请量为: 100.0
agent 23 在 7999 次迭代中的资源申请量为: 100.0
agent 24 在 7999 次迭代中的资源申请量为: 100.0
agent 25 在 7999 次迭代中的资源申请量为: 100.0
agent 26 在 7999 次迭代中的资源申请量为: 100.0
agent 27 在 7999 次迭代中的资源申请量为: 100.0
agent 28 在 7999 次迭代中的资源申请量为: 100.0
agent 29 在 7999

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 100.0
agent 2 在 9999 次迭代中的资源申请量为: 100.0
agent 3 在 9999 次迭代中的资源申请量为: 100.0
agent 4 在 9999 次迭代中的资源申请量为: 100.0
agent 5 在 9999 次迭代中的资源申请量为: 100.0
agent 6 在 9999 次迭代中的资源申请量为: 100.0
agent 7 在 9999 次迭代中的资源申请量为: 100.0
agent 8 在 9999 次迭代中的资源申请量为: 100.0
agent 9 在 9999 次迭代中的资源申请量为: 100.0
agent 10 在 9999 次迭代中的资源申请量为: 100.0
agent 11 在 9999 次迭代中的资源申请量为: 100.0
agent 12 在 9999 次迭代中的资源申请量为: 100.0
agent 13 在 9999 次迭代中的资源申请量为: 100.0
agent 14 在 9999 次迭代中的资源申请量为: 100.0
agent 15 在 9999 次迭代中的资源申请量为: 100.0
agent 16 在 9999 次迭代中的资源申请量为: 100.0
agent 17 在 9999 次迭代中的资源申请量为: 100.0
agent 18 在 9999 次迭代中的资源申请量为: 100.0
agent 19 在 9999 次迭代中的资源申请量为: 100.0
agent 20 在 9999 次迭代中的资源申请量为: 100.0
agent 21 在 9999 次迭代中的资源申请量为: 100.0
agent 22 在 9999 次迭代中的资源申请量为: 100.0
agent 23 在 9999 次迭代中的资源申请量为: 100.0
agent 24 在 9999 次迭代中的资源申请量为: 100.0
agent 25 在 9999 次迭代中的资源申请量为: 100.0
agent 26 在 9999 次迭代中的资源申请量为: 100.0
agent 27 在 9999 次迭代中的资源申请量为: 100.0
agent 28 在 9999 次迭代中的资源申请量为: 100.0
agent 29 在 9999

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 100.0
agent 2 在 51999 次迭代中的资源申请量为: 100.0
agent 3 在 51999 次迭代中的资源申请量为: 100.0
agent 4 在 51999 次迭代中的资源申请量为: 100.0
agent 5 在 51999 次迭代中的资源申请量为: 100.0
agent 6 在 51999 次迭代中的资源申请量为: 100.0
agent 7 在 51999 次迭代中的资源申请量为: 100.0
agent 8 在 51999 次迭代中的资源申请量为: 100.0
agent 9 在 51999 次迭代中的资源申请量为: 100.0
agent 10 在 51999 次迭代中的资源申请量为: 100.0
agent 11 在 51999 次迭代中的资源申请量为: 100.0
agent 12 在 51999 次迭代中的资源申请量为: 100.0
agent 13 在 51999 次迭代中的资源申请量为: 100.0
agent 14 在 51999 次迭代中的资源申请量为: 100.0
agent 15 在 51999 次迭代中的资源申请量为: 100.0
agent 16 在 51999 次迭代中的资源申请量为: 100.0
agent 17 在 51999 次迭代中的资源申请量为: 100.0
agent 18 在 51999 次迭代中的资源申请量为: 100.0
agent 19 在 51999 次迭代中的资源申请量为: 100.0
agent 20 在 51999 次迭代中的资源申请量为: 100.0
agent 21 在 51999 次迭代中的资源申请量为: 100.0
agent 22 在 51999 次迭代中的资源申请量为: 100.0
agent 23 在 51999 次迭代中的资源申请量为: 100.0
agent 24 在 51999 次迭代中的资源申请量为: 100.0
agent 25 在 51999 次迭代中的资源申请量为: 100.0
agent 26 在 51999 次迭代中的资源申请量为: 100.0
agent 27 在 51999 次迭代中的资源申请量为: 100.0
agent 28 在 51999 次迭代中的

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 100.0
agent 2 在 53999 次迭代中的资源申请量为: 100.0
agent 3 在 53999 次迭代中的资源申请量为: 100.0
agent 4 在 53999 次迭代中的资源申请量为: 100.0
agent 5 在 53999 次迭代中的资源申请量为: 100.0
agent 6 在 53999 次迭代中的资源申请量为: 100.0
agent 7 在 53999 次迭代中的资源申请量为: 100.0
agent 8 在 53999 次迭代中的资源申请量为: 100.0
agent 9 在 53999 次迭代中的资源申请量为: 100.0
agent 10 在 53999 次迭代中的资源申请量为: 100.0
agent 11 在 53999 次迭代中的资源申请量为: 100.0
agent 12 在 53999 次迭代中的资源申请量为: 100.0
agent 13 在 53999 次迭代中的资源申请量为: 100.0
agent 14 在 53999 次迭代中的资源申请量为: 100.0
agent 15 在 53999 次迭代中的资源申请量为: 100.0
agent 16 在 53999 次迭代中的资源申请量为: 100.0
agent 17 在 53999 次迭代中的资源申请量为: 100.0
agent 18 在 53999 次迭代中的资源申请量为: 100.0
agent 19 在 53999 次迭代中的资源申请量为: 100.0
agent 20 在 53999 次迭代中的资源申请量为: 100.0
agent 21 在 53999 次迭代中的资源申请量为: 100.0
agent 22 在 53999 次迭代中的资源申请量为: 100.0
agent 23 在 53999 次迭代中的资源申请量为: 100.0
agent 24 在 53999 次迭代中的资源申请量为: 100.0
agent 25 在 53999 次迭代中的资源申请量为: 100.0
agent 26 在 53999 次迭代中的资源申请量为: 100.0
agent 27 在 53999 次迭代中的资源申请量为: 100.0
agent 28 在 53999 次迭代中的

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 100.0
agent 2 在 55999 次迭代中的资源申请量为: 100.0
agent 3 在 55999 次迭代中的资源申请量为: 100.0
agent 4 在 55999 次迭代中的资源申请量为: 100.0
agent 5 在 55999 次迭代中的资源申请量为: 100.0
agent 6 在 55999 次迭代中的资源申请量为: 100.0
agent 7 在 55999 次迭代中的资源申请量为: 100.0
agent 8 在 55999 次迭代中的资源申请量为: 100.0
agent 9 在 55999 次迭代中的资源申请量为: 100.0
agent 10 在 55999 次迭代中的资源申请量为: 100.0
agent 11 在 55999 次迭代中的资源申请量为: 100.0
agent 12 在 55999 次迭代中的资源申请量为: 100.0
agent 13 在 55999 次迭代中的资源申请量为: 100.0
agent 14 在 55999 次迭代中的资源申请量为: 100.0
agent 15 在 55999 次迭代中的资源申请量为: 100.0
agent 16 在 55999 次迭代中的资源申请量为: 100.0
agent 17 在 55999 次迭代中的资源申请量为: 100.0
agent 18 在 55999 次迭代中的资源申请量为: 100.0
agent 19 在 55999 次迭代中的资源申请量为: 100.0
agent 20 在 55999 次迭代中的资源申请量为: 100.0
agent 21 在 55999 次迭代中的资源申请量为: 100.0
agent 22 在 55999 次迭代中的资源申请量为: 100.0
agent 23 在 55999 次迭代中的资源申请量为: 100.0
agent 24 在 55999 次迭代中的资源申请量为: 100.0
agent 25 在 55999 次迭代中的资源申请量为: 100.0
agent 26 在 55999 次迭代中的资源申请量为: 100.0
agent 27 在 55999 次迭代中的资源申请量为: 100.0
agent 28 在 55999 次迭代中的

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 100.0
agent 2 在 57999 次迭代中的资源申请量为: 100.0
agent 3 在 57999 次迭代中的资源申请量为: 100.0
agent 4 在 57999 次迭代中的资源申请量为: 100.0
agent 5 在 57999 次迭代中的资源申请量为: 100.0
agent 6 在 57999 次迭代中的资源申请量为: 100.0
agent 7 在 57999 次迭代中的资源申请量为: 100.0
agent 8 在 57999 次迭代中的资源申请量为: 100.0
agent 9 在 57999 次迭代中的资源申请量为: 100.0
agent 10 在 57999 次迭代中的资源申请量为: 100.0
agent 11 在 57999 次迭代中的资源申请量为: 100.0
agent 12 在 57999 次迭代中的资源申请量为: 100.0
agent 13 在 57999 次迭代中的资源申请量为: 100.0
agent 14 在 57999 次迭代中的资源申请量为: 100.0
agent 15 在 57999 次迭代中的资源申请量为: 100.0
agent 16 在 57999 次迭代中的资源申请量为: 100.0
agent 17 在 57999 次迭代中的资源申请量为: 100.0
agent 18 在 57999 次迭代中的资源申请量为: 100.0
agent 19 在 57999 次迭代中的资源申请量为: 100.0
agent 20 在 57999 次迭代中的资源申请量为: 100.0
agent 21 在 57999 次迭代中的资源申请量为: 100.0
agent 22 在 57999 次迭代中的资源申请量为: 100.0
agent 23 在 57999 次迭代中的资源申请量为: 100.0
agent 24 在 57999 次迭代中的资源申请量为: 100.0
agent 25 在 57999 次迭代中的资源申请量为: 100.0
agent 26 在 57999 次迭代中的资源申请量为: 100.0
agent 27 在 57999 次迭代中的资源申请量为: 100.0
agent 28 在 57999 次迭代中的

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 100.0
agent 2 在 59999 次迭代中的资源申请量为: 100.0
agent 3 在 59999 次迭代中的资源申请量为: 100.0
agent 4 在 59999 次迭代中的资源申请量为: 100.0
agent 5 在 59999 次迭代中的资源申请量为: 100.0
agent 6 在 59999 次迭代中的资源申请量为: 100.0
agent 7 在 59999 次迭代中的资源申请量为: 100.0
agent 8 在 59999 次迭代中的资源申请量为: 100.0
agent 9 在 59999 次迭代中的资源申请量为: 100.0
agent 10 在 59999 次迭代中的资源申请量为: 100.0
agent 11 在 59999 次迭代中的资源申请量为: 100.0
agent 12 在 59999 次迭代中的资源申请量为: 100.0
agent 13 在 59999 次迭代中的资源申请量为: 100.0
agent 14 在 59999 次迭代中的资源申请量为: 100.0
agent 15 在 59999 次迭代中的资源申请量为: 100.0
agent 16 在 59999 次迭代中的资源申请量为: 100.0
agent 17 在 59999 次迭代中的资源申请量为: 100.0
agent 18 在 59999 次迭代中的资源申请量为: 100.0
agent 19 在 59999 次迭代中的资源申请量为: 100.0
agent 20 在 59999 次迭代中的资源申请量为: 100.0
agent 21 在 59999 次迭代中的资源申请量为: 100.0
agent 22 在 59999 次迭代中的资源申请量为: 100.0
agent 23 在 59999 次迭代中的资源申请量为: 100.0
agent 24 在 59999 次迭代中的资源申请量为: 100.0
agent 25 在 59999 次迭代中的资源申请量为: 100.0
agent 26 在 59999 次迭代中的资源申请量为: 100.0
agent 27 在 59999 次迭代中的资源申请量为: 100.0
agent 28 在 59999 次迭代中的

In [8]:
np.savetxt('./data/00_DGD_agent_resource_quantity_min_attack.txt',agent_resource_quantity_attack1,fmt='%0.8f')#用户资源
np.savetxt('./data/00_DGD_average_cost_honest_min_attack.txt',average_cost_honest_attack1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/00_DGD_average_resource_honest_min_attack.txt',average_resource_honest_attack1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/00_DGD_dual_variable_min_attack.txt',dual_variable_attack1,fmt='%0.8f')#用户对偶变量

# DGD算法+大值攻击

In [9]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [10]:
#定义变量 
agent_resource_quantity_attack2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_attack2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_attack2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_attack2 = 0.6
dual_variable_attack2 = np.zeros(shape=(100,60000))#对偶变量
proportion_attack2 = 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack2=拜占庭用户数量
W_attack2 =np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt")
iteration_attack2=[]#绘图横坐标
a_attack2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_attack2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [11]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_attack2*100)):
    dual_variable_attack2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_attack2*100)),100):
    dual_variable_attack2[i][0]=-0.01

#诚实用户资源量的初始化
for i in range (0,100-int(proportion_attack2*100)):
    agent_resource_quantity_attack2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_attack2=0
for i in range (0,100-int(proportion_attack2*100)):
    sum_attack2 = sum_attack2 + agent_resource_quantity_attack2[i][0]
average_resource_honest_attack2[0][0]=sum_attack2/(100-int(proportion_attack2*100))

#诚实用户平均成本初始化
sum_attack2=0
for i in range(0,100-int(proportion_attack2*100)):
    sum_attack2=sum_attack2+a_attack2[i]*(agent_resource_quantity_attack2[i][0]-b_attack2[i])*(agent_resource_quantity_attack2[i][0]-b_attack2[i])
average_cost_honest_attack2[0][0]=sum_attack2/(100-int(proportion_attack2*100))
    
iteration_attack2.append(0)

In [12]:
#DGD算法 
def DGD_attack2(a_attack2,b_attack2,agent_resource_quantity_attack2,average_resource_honest_attack2,average_cost_honest_attack2,iteration_update_attack2,dual_variable_attack2,W_attack2,iteration_attack2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_attack2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_attack2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_attack2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_attack2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_attack2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_attack2.append(dual_variable_attack2[i][k-1]-iteration_update_attack2*((50/100)-(1/100)*agent_resource_quantity_attack2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_attack2*100)),100):
            dual_variable_intermediate_attack2.append(dual_variable_attack2[i][k-1])
        #阶段二：聚合 每个用户聚合自身以及邻居的对偶值   
        for i in range(0,100-int(proportion_attack2*100)):
            mix_sum_dual_attack2=0
            for j in range(0,100):
                if W_attack2[i][j]!=0:
                    mix_sum_dual_attack2=mix_sum_dual_attack2+W_attack2[i][j]*dual_variable_intermediate_attack2[j]
            dual_variable_attack2[i][k]=mix_sum_dual_attack2 
                
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_attack2*100),100):
            dual_variable_attack2[i][k]=dual_variable_attack2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_attack2*100)):
            agent_resource_quantity_attack2[i][k]=(dual_variable_attack2[i][k]-2*a_attack2[i]*b_attack2[i])/(-2*a_attack2[i])
            #原变量的投影操作
            if agent_resource_quantity_attack2[i][k] < 0:
                agent_resource_quantity_attack2[i][k]=0
            elif agent_resource_quantity_attack2[i][k] > 100: 
                agent_resource_quantity_attack2[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_attack2*100),100):
            agent_resource_quantity_attack2[i][k]=agent_resource_quantity_attack2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_attack2=0
        for i in range (0,100-int(proportion_attack2*100)):
            sum1_attack2 = sum1_attack2 + agent_resource_quantity_attack2[i][k]
        average_resource_honest_attack2[0][k]=sum1_attack2/(100-int(proportion_attack2*100))
        
        #计算所有诚实用户的平均成本
        sum2_attack2=0
        for i in range(0,100-int(proportion_attack2*100)):
            sum2_attack2=sum2_attack2+a_attack2[i]*(agent_resource_quantity_attack2[i][k]-b_attack2[i])*(agent_resource_quantity_attack2[i][k]-b_attack2[i])
        average_cost_honest_attack2[0][k]=sum2_attack2/(100-int(proportion_attack2*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_attack2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_attack2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_attack2)
            print("****************************************************************************")
        
        iteration_update_attack2=1/(pow(k+1,0.2)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [13]:
DGD_attack2(a_attack2,b_attack2,agent_resource_quantity_attack2,average_resource_honest_attack2,average_cost_honest_attack2,iteration_update_attack2,dual_variable_attack2,W_attack2,iteration_attack2)

****************************DGD算法+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 3.1186816790686422
agent 2 在 3999 次迭代中的资源申请量为: 3.446999444485879
agent 3 在 3999 次迭代中的资源申请量为: 2.5959142871061824
agent 4 在 3999 次迭代中的资源申请量为: 2.0114045685572486
agent 5 在 3999 次迭代中的资源申请量为: 3.343462675712214
agent 6 在 3999 次迭代中的资源申请量为: 3.3474436489220825
agent 7 在 3999 次迭代中的资源申请量为: 2.278694041501355
agent 8 在 3999 次迭代中的资源申请量为: 2.793363730038825
agent 9 在 3999 次迭代中的资源申请量为: 3.1076109760391137
agent 10 在 3999 次迭代中的资源申请量为: 2.844843018764438
agent 11 在 3999 次迭代中的资源申请量为: 1.5696258153820344
agent 12 在 3999 次迭代中的资源申请量为: 2.2543113239761787
agent 13 在 3999 次迭代中的资源申请量为: 1.7728227753512549
agent 14 在 3999 次迭代中的资源申请量为: 2.851244044137195
agent 15 在 3999 次迭代中的资源申请量为: 2.731279381116682
agent 16 在 3999 次迭代中的资源申请量为: 2.854007707930974
agent 17 在 3999 次迭代中的资源申请量为: 3.327382150876121
agent 18 在 3999 次迭代中的资源申请量为: 2.699416695547149
agent 19 在 3999 次迭代中的资源申请量为: 1.9140387094516391
agent 20 在 3999 次迭代中的资源申请量为: 3.4237750284992754
agent 21 在 3999 次迭代中的资源申请量为: 3.42736503219538

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 3.069470072649317
agent 2 在 5999 次迭代中的资源申请量为: 3.390656037810998
agent 3 在 5999 次迭代中的资源申请量为: 2.53336484666069
agent 4 在 5999 次迭代中的资源申请量为: 1.9765278399215838
agent 5 在 5999 次迭代中的资源申请量为: 3.2901839321484965
agent 6 在 5999 次迭代中的资源申请量为: 3.2954880747267814
agent 7 在 5999 次迭代中的资源申请量为: 2.215077026750401
agent 8 在 5999 次迭代中的资源申请量为: 2.7467520260571012
agent 9 在 5999 次迭代中的资源申请量为: 3.0409102649992907
agent 10 在 5999 次迭代中的资源申请量为: 2.7973568377942293
agent 11 在 5999 次迭代中的资源申请量为: 1.5126659851019022
agent 12 在 5999 次迭代中的资源申请量为: 2.2207441398673957
agent 13 在 5999 次迭代中的资源申请量为: 1.7211320901137417
agent 14 在 5999 次迭代中的资源申请量为: 2.8101001848117053
agent 15 在 5999 次迭代中的资源申请量为: 2.691632135538021
agent 16 在 5999 次迭代中的资源申请量为: 2.811412555429526
agent 17 在 5999 次迭代中的资源申请量为: 3.2818338627643175
agent 18 在 5999 次迭代中的资源申请量为: 2.6519096307090777
agent 19 在 5999 次迭代中的资源申请量为: 1.8678032253696948
agent 20 在 5999 次迭代中的资源申请量为: 3.375880742634416
agent 21 在 5999 次迭代中的资源申请量为: 3.360629194026

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 3.036890141780421
agent 2 在 7999 次迭代中的资源申请量为: 3.3533542896313193
agent 3 在 7999 次迭代中的资源申请量为: 2.491954754058906
agent 4 在 7999 次迭代中的资源申请量为: 1.953438100711517
agent 5 在 7999 次迭代中的资源申请量为: 3.254911335303686
agent 6 在 7999 次迭代中的资源申请量为: 3.2610914824200066
agent 7 在 7999 次迭代中的资源申请量为: 2.1729602228342633
agent 8 在 7999 次迭代中的资源申请量为: 2.7158934528236376
agent 9 在 7999 次迭代中的资源申请量为: 2.996751871003677
agent 10 在 7999 次迭代中的资源申请量为: 2.7659187831695475
agent 11 在 7999 次迭代中的资源申请量为: 1.4749564558999786
agent 12 在 7999 次迭代中的资源申请量为: 2.1985215935560185
agent 13 在 7999 次迭代中的资源申请量为: 1.6869110879070666
agent 14 在 7999 次迭代中的资源申请量为: 2.7828613453346422
agent 15 在 7999 次迭代中的资源申请量为: 2.6653843017095364
agent 16 在 7999 次迭代中的资源申请量为: 2.7832131359177312
agent 17 在 7999 次迭代中的资源申请量为: 3.2516792286558593
agent 18 在 7999 次迭代中的资源申请量为: 2.620458377130109
agent 19 在 7999 次迭代中的资源申请量为: 1.8371938951872586
agent 20 在 7999 次迭代中的资源申请量为: 3.344172814115099
agent 21 在 7999 次迭代中的资源申请量为: 3.31644746985

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 3.012870279887464
agent 2 在 9999 次迭代中的资源申请量为: 3.325853063326924
agent 3 在 9999 次迭代中的资源申请量为: 2.4614247550505532
agent 4 在 9999 次迭代中的资源申请量为: 1.9364149595769764
agent 5 在 9999 次迭代中的资源申请量为: 3.228906252147864
agent 6 在 9999 次迭代中的资源申请量为: 3.2357322389364107
agent 7 在 9999 次迭代中的资源申请量为: 2.14190922661714
agent 8 在 9999 次迭代中的资源申请量为: 2.693142736820808
agent 9 在 9999 次迭代中的资源申请量为: 2.964195665610429
agent 10 在 9999 次迭代中的资源申请量为: 2.7427405688729407
agent 11 在 9999 次迭代中的资源申请量为: 1.4471547756074419
agent 12 在 9999 次迭代中的资源申请量为: 2.1821379082378414
agent 13 在 9999 次迭代中的资源申请量为: 1.6616814085192537
agent 14 在 9999 次迭代中的资源申请量为: 2.762779222532274
agent 15 在 9999 次迭代中的资源申请量为: 2.646032903533306
agent 16 在 9999 次迭代中的资源申请量为: 2.7624229358866246
agent 17 在 9999 次迭代中的资源申请量为: 3.229447466645272
agent 18 在 9999 次迭代中的资源申请量为: 2.597270738089453
agent 19 在 9999 次迭代中的资源申请量为: 1.8146270194064642
agent 20 在 9999 次迭代中的资源申请量为: 3.3207957760418854
agent 21 在 9999 次迭代中的资源申请量为: 3.283874020042057

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 2.864740526785171
agent 2 在 51999 次迭代中的资源申请量为: 3.1562506413342932
agent 3 在 51999 次迭代中的资源申请量为: 2.273146629894209
agent 4 在 51999 次迭代中的资源申请量为: 1.8314336697350853
agent 5 在 51999 次迭代中的资源申请量为: 3.0685333609131504
agent 6 在 51999 次迭代中的资源申请量为: 3.0793418726779853
agent 7 在 51999 次迭代中的资源申请量为: 1.950418741099282
agent 8 在 51999 次迭代中的资源申请量为: 2.5528407425967177
agent 9 在 51999 次迭代中的资源申请量为: 2.763422257178675
agent 10 在 51999 次迭代中的资源申请量为: 2.599797345404434
agent 11 在 51999 次迭代中的资源申请量为: 1.2757036164291133
agent 12 在 51999 次迭代中的资源申请量为: 2.0811020417569264
agent 13 在 51999 次迭代中的资源申请量为: 1.5060919544654752
agent 14 在 51999 次迭代中的资源申请量为: 2.6389327957757893
agent 15 在 51999 次迭代中的资源申请量为: 2.5266946071730176
agent 16 在 51999 次迭代中的资源申请量为: 2.634212103557054
agent 17 在 51999 次迭代中的资源申请量为: 3.092345508733871
agent 18 在 51999 次迭代中的资源申请量为: 2.454274782517618
agent 19 在 51999 次迭代中的资源申请量为: 1.6754602224572224
agent 20 在 51999 次迭代中的资源申请量为: 3.176628757504936
agent 21 在 51999 次迭代中的资

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 2.861882190729522
agent 2 在 53999 次迭代中的资源申请量为: 3.1529779089257635
agent 3 在 53999 次迭代中的资源申请量为: 2.2695135723707622
agent 4 在 53999 次迭代中的资源申请量为: 1.8294079332885482
agent 5 在 53999 次迭代中的资源申请量为: 3.065438775423704
agent 6 在 53999 次迭代中的资源申请量为: 3.076324125691143
agent 7 在 53999 次迭代中的资源申请量为: 1.9467237077773907
agent 8 在 53999 次迭代中的资源申请量为: 2.5501334674258316
agent 9 在 53999 次迭代中的资源申请量为: 2.759548095049788
agent 10 在 53999 次迭代中的资源申请量为: 2.5970390246458948
agent 11 在 53999 次迭代中的资源申请量为: 1.272395287554352
agent 12 在 53999 次迭代中的资源申请量为: 2.079152467842566
agent 13 在 53999 次迭代中的资源申请量为: 1.5030896925356736
agent 14 在 53999 次迭代中的资源申请量为: 2.636543024712279
agent 15 在 53999 次迭代中的资源申请量为: 2.5243918553741924
agent 16 在 53999 次迭代中的资源申请量为: 2.6317381552916843
agent 17 在 53999 次迭代中的资源申请量为: 3.0896999789986275
agent 18 在 53999 次迭代中的资源申请量为: 2.451515531627844
agent 19 在 53999 次迭代中的资源申请量为: 1.6727748758296377
agent 20 在 53999 次迭代中的资源申请量为: 3.173846858252788
agent 21 在 53999 次迭代中的资

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 2.859147888442783
agent 2 在 55999 次迭代中的资源申请量为: 3.1498471904402736
agent 3 在 55999 次迭代中的资源申请量为: 2.2660381661679367
agent 4 在 55999 次迭代中的资源申请量为: 1.8274701010607035
agent 5 在 55999 次迭代中的资源申请量为: 3.062478475347437
agent 6 在 55999 次迭代中的资源申请量为: 3.0734373294245283
agent 7 在 55999 次迭代中的资源申请量为: 1.9431890155057618
agent 8 在 55999 次迭代中的资源申请量为: 2.54754367143998
agent 9 在 55999 次迭代中的资源申请量为: 2.755842046931997
agent 10 在 55999 次迭代中的资源申请量为: 2.594400395348835
agent 11 在 55999 次迭代中的资源申请量为: 1.2692305201468912
agent 12 在 55999 次迭代中的资源申请量为: 2.0772874942365878
agent 13 在 55999 次迭代中的资源申请量为: 1.5002177104177912
agent 14 在 55999 次迭代中的资源申请量为: 2.6342569542924203
agent 15 在 55999 次迭代中的资源申请量为: 2.522189029171898
agent 16 在 55999 次迭代中的资源申请量为: 2.6293715618229
agent 17 在 55999 次迭代中的资源申请量为: 3.087169249041396
agent 18 在 55999 次迭代中的资源申请量为: 2.4488760155682607
agent 19 在 55999 次迭代中的资源申请量为: 1.6702060576360758
agent 20 在 55999 次迭代中的资源申请量为: 3.1711856748140845
agent 21 在 55999 次迭代中的资源申

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 2.8565280878110637
agent 2 在 57999 次迭代中的资源申请量为: 3.1468475720676325
agent 3 在 57999 次迭代中的资源申请量为: 2.2627082956516205
agent 4 在 57999 次迭代中的资源申请量为: 1.8256134175281713
agent 5 在 57999 次迭代中的资源申请量为: 3.0596421407277403
agent 6 在 57999 次迭代中的资源申请量为: 3.0706714202118874
agent 7 在 57999 次迭代中的资源申请量为: 1.9398023419242985
agent 8 在 57999 次迭代中的资源申请量为: 2.545062326226009
agent 9 在 57999 次迭代中的资源申请量为: 2.752291193111994
agent 10 在 57999 次迭代中的资源申请量为: 2.5918722592193983
agent 11 在 57999 次迭代中的资源申请量为: 1.2661982813338761
agent 12 在 57999 次迭代中的资源申请量为: 2.0755006192721943
agent 13 在 57999 次迭代中的资源申请量为: 1.4974659959542442
agent 14 在 57999 次迭代中的资源申请量为: 2.6320666150395327
agent 15 在 57999 次迭代中的资源申请量为: 2.5200784491553425
agent 16 在 57999 次迭代中的资源申请量为: 2.6271040727825556
agent 17 在 57999 次迭代中的资源申请量为: 3.084744496368331
agent 18 在 57999 次迭代中的资源申请量为: 2.4463470325651167
agent 19 在 57999 次迭代中的资源申请量为: 1.6677448124898608
agent 20 在 57999 次迭代中的资源申请量为: 3.168635930112472
agent 21 在 57999 次

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 2.8540142867366356
agent 2 在 59999 次迭代中的资源申请量为: 3.1439693193089404
agent 3 在 59999 次迭代中的资源申请量为: 2.259513154371023
agent 4 在 59999 次迭代中的资源申请量为: 1.8238318571522893
agent 5 在 59999 次迭代中的资源申请量为: 3.0569205667558768
agent 6 在 59999 次迭代中的资源申请量为: 3.0680174218396536
agent 7 在 59999 次迭代中的资源申请量为: 1.9365526961959123
agent 8 在 59999 次迭代中的资源申请量为: 2.5426813789603773
agent 9 在 59999 次迭代中的资源申请量为: 2.7488840099473624
agent 10 在 59999 次迭代中的资源申请量为: 2.589446411899395
agent 11 在 59999 次迭代中的资源申请量为: 1.2632887304294718
agent 12 在 59999 次迭代中的资源申请量为: 2.0737860438404216
agent 13 在 59999 次迭代中的资源申请量为: 1.4948256188783964
agent 14 在 59999 次迭代中的资源申请量为: 2.629964898637114
agent 15 在 59999 次迭代中的资源申请量为: 2.5180532657341232
agent 16 在 59999 次迭代中的资源申请量为: 2.624928329320345
agent 17 在 59999 次迭代中的资源申请量为: 3.0824178518235867
agent 18 在 59999 次迭代中的资源申请量为: 2.4439203751682337
agent 19 在 59999 次迭代中的资源申请量为: 1.6653831527060854
agent 20 在 59999 次迭代中的资源申请量为: 3.166189349525272
agent 21 在 59999 次迭

In [14]:
np.savetxt('./data/00_DGD_agent_resource_quantity_max_attack.txt',agent_resource_quantity_attack2,fmt='%0.8f')#用户资源
np.savetxt('./data/00_DGD_average_cost_honest_max_attack.txt',average_cost_honest_attack2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/00_DGD_average_resource_honest_max_attack.txt',average_resource_honest_attack2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/00_DGD_dual_variable_max_attack.txt',dual_variable_attack2,fmt='%0.8f')#用户对偶变量

# DGD算法+高斯攻击1 ——大值高斯攻击

In [15]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [16]:
#定义变量 
agent_resource_quantity_attack3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_attack3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_attack3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_attack3 = 0.6
dual_variable_attack3 = np.zeros(shape=(100,60000))#对偶变量
proportion_attack3 = 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
W_attack3 =np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt")
iteration_attack3=[]#绘图横坐标
random_attack3=np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt") 
a_attack3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_attack3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [17]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_attack3*100)):
    dual_variable_attack3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_attack3*100)),100):
    dual_variable_attack3[i][0]=random_attack3[i-94][0]

#诚实用户资源量的初始化
for i in range (0,100-int(proportion_attack3*100)):
    agent_resource_quantity_attack3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_attack3=0
for i in range (0,100-int(proportion_attack3*100)):
    sum_attack3 = sum_attack3 + agent_resource_quantity_attack3[i][0]
average_resource_honest_attack3[0][0]=sum_attack3/(100-int(proportion_attack3*100))

#诚实用户平均成本初始化
sum_attack3=0
for i in range(0,100-int(proportion_attack3*100)):
    sum_attack3=sum_attack3+a_attack3[i]*(agent_resource_quantity_attack3[i][0]-b_attack3[i])*(agent_resource_quantity_attack3[i][0]-b_attack3[i])
average_cost_honest_attack3[0][0]=sum_attack3/(100-int(proportion_attack3*100))
    
iteration_attack3.append(0)

In [18]:
#DSubGD算法 
def DGD_attack3(a_attack3,b_attack3,agent_resource_quantity_attack3,average_resource_honest_attack3,average_cost_honest_attack3,iteration_update_attack3,dual_variable_attack3,W_attack3,iteration_attack3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_attack3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_attack3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_attack3.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_attack3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_attack3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_attack3.append(dual_variable_attack3[i][k-1]-iteration_update_attack3*((50/100)-(1/100)*agent_resource_quantity_attack3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_attack3*100)),100):
            dual_variable_intermediate_attack3.append(dual_variable_attack3[i][k-1])
        #阶段二：聚合 每个用户聚合自身以及邻居的对偶值   
        for i in range(0,100-int(proportion_attack3*100)):
            mix_sum_dual_attack3=0
            for j in range(0,100):
                if W_attack3[i][j]!=0:
                    mix_sum_dual_attack3=mix_sum_dual_attack3+W_attack3[i][j]*dual_variable_intermediate_attack3[j]
            dual_variable_attack3[i][k]=mix_sum_dual_attack3
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_attack3*100),100):
            dual_variable_attack3[i][k]=random_attack3[i-94][k] #服从高斯分布的攻击
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_attack3*100)):
            agent_resource_quantity_attack3[i][k]=(dual_variable_attack3[i][k]-2*a_attack3[i]*b_attack3[i])/(-2*a_attack3[i])
            #原变量的投影操作
            if agent_resource_quantity_attack3[i][k] < 0:
                agent_resource_quantity_attack3[i][k]=0
            elif agent_resource_quantity_attack3[i][k] > 100: 
                agent_resource_quantity_attack3[i][k]=100 
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_attack3*100),100):
            agent_resource_quantity_attack3[i][k]=agent_resource_quantity_attack3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_attack3=0
        for i in range (0,100-int(proportion_attack3*100)):
            sum1_attack3 = sum1_attack3 + agent_resource_quantity_attack3[i][k]
        average_resource_honest_attack3[0][k]=sum1_attack3/(100-int(proportion_attack3*100))
        
        #计算所有诚实用户的平均成本
        sum2_attack3=0
        for i in range(0,100-int(proportion_attack3*100)):
            sum2_attack3=sum2_attack3+a_attack3[i]*(agent_resource_quantity_attack3[i][k]-b_attack3[i])*(agent_resource_quantity_attack3[i][k]-b_attack3[i])
        average_cost_honest_attack3[0][k]=sum2_attack3/(100-int(proportion_attack3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_attack3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_attack3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_attack3)
            print("****************************************************************************")
        
        iteration_update_attack3=1/(pow(k+1,0.2)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [19]:
DGD_attack3(a_attack3,b_attack3,agent_resource_quantity_attack3,average_resource_honest_attack3,average_cost_honest_attack3,iteration_update_attack3,dual_variable_attack3,W_attack3,iteration_attack3)

****************************DGD算法+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 15.258088487580064
agent 2 在 3999 次迭代中的资源申请量为: 15.28308458192215
agent 3 在 3999 次迭代中的资源申请量为: 15.70037219162227
agent 4 在 3999 次迭代中的资源申请量为: 10.146049160814236
agent 5 在 3999 次迭代中的资源申请量为: 15.647391613245187
agent 6 在 3999 次迭代中的资源申请量为: 15.169404915908357
agent 7 在 3999 次迭代中的资源申请量为: 16.670659799653038
agent 8 在 3999 次迭代中的资源申请量为: 12.604481927039098
agent 9 在 3999 次迭代中的资源申请量为: 17.187136542724254
agent 10 在 3999 次迭代中的资源申请量为: 14.381717794369283
agent 11 在 3999 次迭代中的资源申请量为: 16.20140551564903
agent 12 在 3999 次迭代中的资源申请量为: 10.488494357933345
agent 13 在 3999 次迭代中的资源申请量为: 12.566873843181602
agent 14 在 3999 次迭代中的资源申请量为: 11.500921239101052
agent 15 在 3999 次迭代中的资源申请量为: 11.7494203673957
agent 16 在 3999 次迭代中的资源申请量为: 13.526803567004087
agent 17 在 3999 次迭代中的资源申请量为: 13.865333000772724
agent 18 在 3999 次迭代中的资源申请量为: 12.519631705181622
agent 19 在 3999 次迭代中的资源申请量为: 11.59673023487521
agent 20 在 3999 次迭代中的资源申请量为: 14.103601450748231
agent 21 在 3999 次迭代中的资源申请量为: 18.261309808

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 14.659936124802819
agent 2 在 5999 次迭代中的资源申请量为: 15.093057218426074
agent 3 在 5999 次迭代中的资源申请量为: 15.455536515978945
agent 4 在 5999 次迭代中的资源申请量为: 9.9048020119826
agent 5 在 5999 次迭代中的资源申请量为: 15.275820236190492
agent 6 在 5999 次迭代中的资源申请量为: 14.618912511470587
agent 7 在 5999 次迭代中的资源申请量为: 16.21368978443854
agent 8 在 5999 次迭代中的资源申请量为: 12.403532145523975
agent 9 在 5999 次迭代中的资源申请量为: 16.969484314767726
agent 10 在 5999 次迭代中的资源申请量为: 14.206252202892186
agent 11 在 5999 次迭代中的资源申请量为: 15.305336950805994
agent 12 在 5999 次迭代中的资源申请量为: 10.031003287511146
agent 13 在 5999 次迭代中的资源申请量为: 12.397320084083992
agent 14 在 5999 次迭代中的资源申请量为: 11.352187037401567
agent 15 在 5999 次迭代中的资源申请量为: 11.47147812784205
agent 16 在 5999 次迭代中的资源申请量为: 12.85550807954581
agent 17 在 5999 次迭代中的资源申请量为: 13.656604931678304
agent 18 在 5999 次迭代中的资源申请量为: 12.405907246258405
agent 19 在 5999 次迭代中的资源申请量为: 11.386330635808141
agent 20 在 5999 次迭代中的资源申请量为: 13.809823520349022
agent 21 在 5999 次迭代中的资源申请量为: 18.236861196

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 14.923580671333507
agent 2 在 7999 次迭代中的资源申请量为: 15.158381779544953
agent 3 在 7999 次迭代中的资源申请量为: 15.488208122554878
agent 4 在 7999 次迭代中的资源申请量为: 10.050092330178229
agent 5 在 7999 次迭代中的资源申请量为: 15.432557713405966
agent 6 在 7999 次迭代中的资源申请量为: 14.717961630579868
agent 7 在 7999 次迭代中的资源申请量为: 16.379108553601743
agent 8 在 7999 次迭代中的资源申请量为: 12.485034056624771
agent 9 在 7999 次迭代中的资源申请量为: 17.040249439785626
agent 10 在 7999 次迭代中的资源申请量为: 13.987667942783684
agent 11 在 7999 次迭代中的资源申请量为: 15.573675748910015
agent 12 在 7999 次迭代中的资源申请量为: 10.204696186914902
agent 13 在 7999 次迭代中的资源申请量为: 12.438390217549635
agent 14 在 7999 次迭代中的资源申请量为: 11.43756209441813
agent 15 在 7999 次迭代中的资源申请量为: 11.572534380405472
agent 16 在 7999 次迭代中的资源申请量为: 12.973106691360137
agent 17 在 7999 次迭代中的资源申请量为: 13.674407297799899
agent 18 在 7999 次迭代中的资源申请量为: 12.48716136616649
agent 19 在 7999 次迭代中的资源申请量为: 11.428676936578642
agent 20 在 7999 次迭代中的资源申请量为: 14.022268382956026
agent 21 在 7999 次迭代中的资源申请量为: 17.85908

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 14.691775158312652
agent 2 在 9999 次迭代中的资源申请量为: 14.90638117093985
agent 3 在 9999 次迭代中的资源申请量为: 15.218010348618009
agent 4 在 9999 次迭代中的资源申请量为: 9.890413625629716
agent 5 在 9999 次迭代中的资源申请量为: 15.280578403400728
agent 6 在 9999 次迭代中的资源申请量为: 14.792848641345746
agent 7 在 9999 次迭代中的资源申请量为: 16.25362163249691
agent 8 在 9999 次迭代中的资源申请量为: 12.305777552196234
agent 9 在 9999 次迭代中的资源申请量为: 16.74867436636781
agent 10 在 9999 次迭代中的资源申请量为: 14.06731230502731
agent 11 在 9999 次迭代中的资源申请量为: 15.778447918045506
agent 12 在 9999 次迭代中的资源申请量为: 10.136428388566516
agent 13 在 9999 次迭代中的资源申请量为: 12.205650924663466
agent 14 在 9999 次迭代中的资源申请量为: 11.265351296599029
agent 15 在 9999 次迭代中的资源申请量为: 11.469978417070285
agent 16 在 9999 次迭代中的资源申请量为: 13.197866468446481
agent 17 在 9999 次迭代中的资源申请量为: 13.637949100534305
agent 18 在 9999 次迭代中的资源申请量为: 12.269349318583064
agent 19 在 9999 次迭代中的资源申请量为: 11.254908582552419
agent 20 在 9999 次迭代中的资源申请量为: 13.667427675529266
agent 21 在 9999 次迭代中的资源申请量为: 17.82512729

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 14.637364551626277
agent 2 在 51999 次迭代中的资源申请量为: 14.920239096432566
agent 3 在 51999 次迭代中的资源申请量为: 15.256420533678922
agent 4 在 51999 次迭代中的资源申请量为: 9.92481161952699
agent 5 在 51999 次迭代中的资源申请量为: 15.305296556204901
agent 6 在 51999 次迭代中的资源申请量为: 14.661381620518423
agent 7 在 51999 次迭代中的资源申请量为: 16.349327462225613
agent 8 在 51999 次迭代中的资源申请量为: 12.29259792565912
agent 9 在 51999 次迭代中的资源申请量为: 16.756058019566176
agent 10 在 51999 次迭代中的资源申请量为: 14.285244057332976
agent 11 在 51999 次迭代中的资源申请量为: 15.595450954058567
agent 12 在 51999 次迭代中的资源申请量为: 9.979312791489289
agent 13 在 51999 次迭代中的资源申请量为: 12.195532453497115
agent 14 在 51999 次迭代中的资源申请量为: 11.271991552114274
agent 15 在 51999 次迭代中的资源申请量为: 11.473954573027559
agent 16 在 51999 次迭代中的资源申请量为: 13.129236231268504
agent 17 在 51999 次迭代中的资源申请量为: 13.74254180176922
agent 18 在 51999 次迭代中的资源申请量为: 12.258635508695656
agent 19 在 51999 次迭代中的资源申请量为: 11.239934482012758
agent 20 在 51999 次迭代中的资源申请量为: 13.604490112451995
agent 21 在 51999 次迭

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 14.607421879095652
agent 2 在 53999 次迭代中的资源申请量为: 14.903373819673657
agent 3 在 53999 次迭代中的资源申请量为: 15.232687355873773
agent 4 在 53999 次迭代中的资源申请量为: 9.813716480827331
agent 5 在 53999 次迭代中的资源申请量为: 15.092771706209257
agent 6 在 53999 次迭代中的资源申请量为: 14.722820066573048
agent 7 在 53999 次迭代中的资源申请量为: 16.129109226551723
agent 8 在 53999 次迭代中的资源申请量为: 12.324001320717183
agent 9 在 53999 次迭代中的资源申请量为: 16.71775285830376
agent 10 在 53999 次迭代中的资源申请量为: 14.288552130432109
agent 11 在 53999 次迭代中的资源申请量为: 15.502588567881183
agent 12 在 53999 次迭代中的资源申请量为: 10.125918465083446
agent 13 在 53999 次迭代中的资源申请量为: 12.222832481881817
agent 14 在 53999 次迭代中的资源申请量为: 11.278795516516025
agent 15 在 53999 次迭代中的资源申请量为: 11.364680071080857
agent 16 在 53999 次迭代中的资源申请量为: 13.015642544899027
agent 17 在 53999 次迭代中的资源申请量为: 13.510374116255553
agent 18 在 53999 次迭代中的资源申请量为: 12.283858936455793
agent 19 在 53999 次迭代中的资源申请量为: 11.265957079952699
agent 20 在 53999 次迭代中的资源申请量为: 13.730658470872717
agent 21 在 53999

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 14.526107902457246
agent 2 在 55999 次迭代中的资源申请量为: 14.776369693713253
agent 3 在 55999 次迭代中的资源申请量为: 15.080150821391454
agent 4 在 55999 次迭代中的资源申请量为: 9.622392363855667
agent 5 在 55999 次迭代中的资源申请量为: 14.818428963429918
agent 6 在 55999 次迭代中的资源申请量为: 14.698178465949109
agent 7 在 55999 次迭代中的资源申请量为: 16.166419075817807
agent 8 在 55999 次迭代中的资源申请量为: 12.171258127779621
agent 9 在 55999 次迭代中的资源申请量为: 16.56991334290027
agent 10 在 55999 次迭代中的资源申请量为: 13.52201997769935
agent 11 在 55999 次迭代中的资源申请量为: 15.33103127688702
agent 12 在 55999 次迭代中的资源申请量为: 10.26367976556097
agent 13 在 55999 次迭代中的资源申请量为: 12.107230320027975
agent 14 在 55999 次迭代中的资源申请量为: 11.125706859206193
agent 15 在 55999 次迭代中的资源申请量为: 11.1555514677952
agent 16 在 55999 次迭代中的资源申请量为: 13.158864316008946
agent 17 在 55999 次迭代中的资源申请量为: 13.59459404531736
agent 18 在 55999 次迭代中的资源申请量为: 12.131538241085122
agent 19 在 55999 次迭代中的资源申请量为: 11.159834907693043
agent 20 在 55999 次迭代中的资源申请量为: 13.779996673516415
agent 21 在 55999 次迭代中的

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 14.651232746857861
agent 2 在 57999 次迭代中的资源申请量为: 15.086380452123018
agent 3 在 57999 次迭代中的资源申请量为: 15.38038293343018
agent 4 在 57999 次迭代中的资源申请量为: 9.90674407616503
agent 5 在 57999 次迭代中的资源申请量为: 15.224754700641332
agent 6 在 57999 次迭代中的资源申请量为: 14.72494419249263
agent 7 在 57999 次迭代中的资源申请量为: 16.08836844470263
agent 8 在 57999 次迭代中的资源申请量为: 12.407938260298732
agent 9 在 57999 次迭代中的资源申请量为: 16.900310366364558
agent 10 在 57999 次迭代中的资源申请量为: 14.184132946892195
agent 11 在 57999 次迭代中的资源申请量为: 15.60386575916861
agent 12 在 57999 次迭代中的资源申请量为: 10.073525413258027
agent 13 在 57999 次迭代中的资源申请量为: 12.319695523718712
agent 14 在 57999 次迭代中的资源申请量为: 11.301727777627686
agent 15 在 57999 次迭代中的资源申请量为: 11.462648413294824
agent 16 在 57999 次迭代中的资源申请量为: 12.925840464405844
agent 17 在 57999 次迭代中的资源申请量为: 13.842815655435672
agent 18 在 57999 次迭代中的资源申请量为: 12.308631459825556
agent 19 在 57999 次迭代中的资源申请量为: 11.408153400789633
agent 20 在 57999 次迭代中的资源申请量为: 13.723051371679231
agent 21 在 57999 次迭代

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 14.749834110812028
agent 2 在 59999 次迭代中的资源申请量为: 14.862688545477239
agent 3 在 59999 次迭代中的资源申请量为: 15.183533651988252
agent 4 在 59999 次迭代中的资源申请量为: 9.93108988598075
agent 5 在 59999 次迭代中的资源申请量为: 15.249641087404424
agent 6 在 59999 次迭代中的资源申请量为: 14.627252182706147
agent 7 在 59999 次迭代中的资源申请量为: 16.056695816092027
agent 8 在 59999 次迭代中的资源申请量为: 12.264350703660421
agent 9 在 59999 次迭代中的资源申请量为: 16.660765121501814
agent 10 在 59999 次迭代中的资源申请量为: 14.07620202050072
agent 11 在 59999 次迭代中的资源申请量为: 15.646193382509361
agent 12 在 59999 次迭代中的资源申请量为: 10.074783867961454
agent 13 在 59999 次迭代中的资源申请量为: 12.167055620591418
agent 14 在 59999 次迭代中的资源申请量为: 11.219441329656958
agent 15 在 59999 次迭代中的资源申请量为: 11.472562506976013
agent 16 在 59999 次迭代中的资源申请量为: 12.930123095502324
agent 17 在 59999 次迭代中的资源申请量为: 13.5863672170554
agent 18 在 59999 次迭代中的资源申请量为: 12.212673259195816
agent 19 在 59999 次迭代中的资源申请量为: 11.238666527318413
agent 20 在 59999 次迭代中的资源申请量为: 13.66674017289289
agent 21 在 59999 次迭代

In [20]:
np.savetxt('./data/00_DGD_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_attack3,fmt='%0.8f')#用户资源
np.savetxt('./data/00_DGD_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_attack3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/00_DGD_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_attack3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/00_DGD_dual_variable_Gaussian_attack_large.txt',dual_variable_attack3,fmt='%0.8f')#用户对偶变量

# DGD算法+高斯攻击2——小值高斯攻击

In [21]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [22]:
#定义变量 
agent_resource_quantity_attack4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_attack4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_attack4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_attack4 = 0.6
dual_variable_attack4 = np.zeros(shape=(100,60000))#对偶变量
proportion_attack4 = 0.06 #100个用户中拜占庭用户的比例 100*proportion_attack1=拜占庭用户数量
W_attack4 =np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt")
iteration_attack4=[]#绘图横坐标
random_attack4=np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt") 
a_attack4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_attack4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数

In [23]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_attack4*100)):
    dual_variable_attack4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_attack4*100)),100):
    dual_variable_attack4[i][0]=random_attack4[i-94][0]

#诚实用户资源量的初始化
for i in range (0,100-int(proportion_attack4*100)):
    agent_resource_quantity_attack4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_attack4=0
for i in range (0,100-int(proportion_attack4*100)):
    sum_attack4 = sum_attack4 + agent_resource_quantity_attack4[i][0]
average_resource_honest_attack4[0][0]=sum_attack4/(100-int(proportion_attack4*100))

#诚实用户平均成本初始化
sum_attack4=0
for i in range(0,100-int(proportion_attack4*100)):
    sum_attack4=sum_attack4+a_attack4[i]*(agent_resource_quantity_attack4[i][0]-b_attack4[i])*(agent_resource_quantity_attack4[i][0]-b_attack4[i])
average_cost_honest_attack4[0][0]=sum_attack4/(100-int(proportion_attack4*100))
    
iteration_attack4.append(0)

In [24]:
#DSubGD算法 
def DGD_attack4(a_attack4,b_attack4,agent_resource_quantity_attack4,average_resource_honest_attack4,average_cost_honest_attack4,iteration_update_attack4,dual_variable_attack4,W_attack4,iteration_attack4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_attack4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_attack4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_attack4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_attack4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_attack4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_attack4.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_attack4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_attack4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_attack4.append(dual_variable_attack4[i][k-1]-iteration_update_attack4*((50/100)-(1/100)*agent_resource_quantity_attack4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_attack4*100)),100):
            dual_variable_intermediate_attack4.append(dual_variable_attack4[i][k-1])
        #阶段二：聚合 每个用户聚合自身以及邻居的对偶值   
        for i in range(0,100-int(proportion_attack4*100)):
            mix_sum_dual_attack4=0
            for j in range(0,100):
                if W_attack4[i][j]!=0:
                    mix_sum_dual_attack4=mix_sum_dual_attack4+W_attack4[i][j]*dual_variable_intermediate_attack4[j]
            dual_variable_attack4[i][k]=mix_sum_dual_attack4
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_attack4*100),100):
            dual_variable_attack4[i][k]=random_attack4[i-94][k] #服从高斯分布的攻击
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_attack4*100)):
            agent_resource_quantity_attack4[i][k]=(dual_variable_attack4[i][k]-2*a_attack4[i]*b_attack4[i])/(-2*a_attack4[i])
            #原变量的投影操作
            if agent_resource_quantity_attack4[i][k] < 0:
                agent_resource_quantity_attack4[i][k]=0
            elif agent_resource_quantity_attack4[i][k] > 100: 
                agent_resource_quantity_attack4[i][k]=100 
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_attack4*100),100):
            agent_resource_quantity_attack4[i][k]=agent_resource_quantity_attack4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_attack4=0
        for i in range (0,100-int(proportion_attack4*100)):
            sum1_attack4 = sum1_attack4 + agent_resource_quantity_attack4[i][k]
        average_resource_honest_attack4[0][k]=sum1_attack4/(100-int(proportion_attack4*100))
        
        #计算所有诚实用户的平均成本
        sum2_attack4=0
        for i in range(0,100-int(proportion_attack4*100)):
            sum2_attack4=sum2_attack4+a_attack4[i]*(agent_resource_quantity_attack4[i][k]-b_attack4[i])*(agent_resource_quantity_attack4[i][k]-b_attack4[i])
        average_cost_honest_attack4[0][k]=sum2_attack4/(100-int(proportion_attack4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_attack4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_attack4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_attack4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_attack4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_attack4)
            print("****************************************************************************")
        
        iteration_update_attack4=1/(pow(k+1,0.2)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [25]:
DGD_attack4(a_attack4,b_attack4,agent_resource_quantity_attack4,average_resource_honest_attack4,average_cost_honest_attack4,iteration_update_attack4,dual_variable_attack4,W_attack4,iteration_attack4)

****************************DGD算法+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资源申请量

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 100.0
agent 2 在 3999 次迭代中的资源申请量为: 100.0
agent 3 在 3999 次迭代中的资源申请量为: 100.0
agent 4 在 3999 次迭代中的资源申请量为: 81.2795369901958
agent 5 在 3999 次迭代中的资源申请量为: 100.0
agent 6 在 3999 次迭代中的资源申请量为: 100.0
agent 7 在 3999 次迭代中的资源申请量为: 100.0
agent 8 在 3999 次迭代中的资源申请量为: 100.0
agent 9 在 3999 次迭代中的资源申请量为: 100.0
agent 10 在 3999 次迭代中的资源申请量为: 100.0
agent 11 在 3999 次迭代中的资源申请量为: 100.0
agent 12 在 3999 次迭代中的资源申请量为: 83.13619110183213
agent 13 在 3999 次迭代中的资源申请量为: 100.0
agent 14 在 3999 次迭代中的资源申请量为: 88.78274467962017
agent 15 在 3999 次迭代中的资源申请量为: 90.58545566013963
agent 16 在 3999 次迭代中的资源申请量为: 100.0
agent 17 在 3999 次迭代中的资源申请量为: 100.0
agent 18 在 3999 次迭代中的资源申请量为: 100.0
agent 19 在 3999 次迭代中的资源申请量为: 98.04249002670588
agent 20 在 3999 次迭代中的资源申请量为: 100.0
agent 21 在 3999 次迭代中的资源申请量为: 100.0
agent 22 在 3999 次迭代中的资源申请量为: 100.0
agent 23 在 3999 次迭代中的资源申请量为: 100.0
agent 24 在 3999 次迭代中的资源申请量为: 87.56836722934882
agent 25 在 3999 次迭代中的资源申请量为: 100.0
agent 26 在 3999 次迭代中的资源申请量为: 100.0
agent 27 在 399

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 100.0
agent 2 在 5999 次迭代中的资源申请量为: 100.0
agent 3 在 5999 次迭代中的资源申请量为: 100.0
agent 4 在 5999 次迭代中的资源申请量为: 82.34856207278341
agent 5 在 5999 次迭代中的资源申请量为: 100.0
agent 6 在 5999 次迭代中的资源申请量为: 100.0
agent 7 在 5999 次迭代中的资源申请量为: 100.0
agent 8 在 5999 次迭代中的资源申请量为: 99.50427411743792
agent 9 在 5999 次迭代中的资源申请量为: 100.0
agent 10 在 5999 次迭代中的资源申请量为: 100.0
agent 11 在 5999 次迭代中的资源申请量为: 100.0
agent 12 在 5999 次迭代中的资源申请量为: 82.61861459180359
agent 13 在 5999 次迭代中的资源申请量为: 100.0
agent 14 在 5999 次迭代中的资源申请量为: 88.50411903115042
agent 15 在 5999 次迭代中的资源申请量为: 91.75948748917561
agent 16 在 5999 次迭代中的资源申请量为: 100.0
agent 17 在 5999 次迭代中的资源申请量为: 100.0
agent 18 在 5999 次迭代中的资源申请量为: 100.0
agent 19 在 5999 次迭代中的资源申请量为: 96.89633090708757
agent 20 在 5999 次迭代中的资源申请量为: 100.0
agent 21 在 5999 次迭代中的资源申请量为: 100.0
agent 22 在 5999 次迭代中的资源申请量为: 100.0
agent 23 在 5999 次迭代中的资源申请量为: 100.0
agent 24 在 5999 次迭代中的资源申请量为: 84.39387990045779
agent 25 在 5999 次迭代中的资源申请量为: 100.0
agent 26 在 5999 次迭代中的资源申请量为: 100.0
a

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 100.0
agent 2 在 7999 次迭代中的资源申请量为: 100.0
agent 3 在 7999 次迭代中的资源申请量为: 100.0
agent 4 在 7999 次迭代中的资源申请量为: 82.94271922327248
agent 5 在 7999 次迭代中的资源申请量为: 100.0
agent 6 在 7999 次迭代中的资源申请量为: 100.0
agent 7 在 7999 次迭代中的资源申请量为: 100.0
agent 8 在 7999 次迭代中的资源申请量为: 99.57556166882945
agent 9 在 7999 次迭代中的资源申请量为: 100.0
agent 10 在 7999 次迭代中的资源申请量为: 100.0
agent 11 在 7999 次迭代中的资源申请量为: 100.0
agent 12 在 7999 次迭代中的资源申请量为: 82.15314477993547
agent 13 在 7999 次迭代中的资源申请量为: 100.0
agent 14 在 7999 次迭代中的资源申请量为: 88.48708368340426
agent 15 在 7999 次迭代中的资源申请量为: 92.04381971319496
agent 16 在 7999 次迭代中的资源申请量为: 100.0
agent 17 在 7999 次迭代中的资源申请量为: 100.0
agent 18 在 7999 次迭代中的资源申请量为: 100.0
agent 19 在 7999 次迭代中的资源申请量为: 97.26200079965629
agent 20 在 7999 次迭代中的资源申请量为: 100.0
agent 21 在 7999 次迭代中的资源申请量为: 100.0
agent 22 在 7999 次迭代中的资源申请量为: 100.0
agent 23 在 7999 次迭代中的资源申请量为: 100.0
agent 24 在 7999 次迭代中的资源申请量为: 85.25751509015133
agent 25 在 7999 次迭代中的资源申请量为: 100.0
agent 26 在 7999 次迭代中的资源申请量为: 100.0
a

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 100.0
agent 2 在 9999 次迭代中的资源申请量为: 100.0
agent 3 在 9999 次迭代中的资源申请量为: 100.0
agent 4 在 9999 次迭代中的资源申请量为: 80.80489882789523
agent 5 在 9999 次迭代中的资源申请量为: 100.0
agent 6 在 9999 次迭代中的资源申请量为: 100.0
agent 7 在 9999 次迭代中的资源申请量为: 100.0
agent 8 在 9999 次迭代中的资源申请量为: 98.59465443051333
agent 9 在 9999 次迭代中的资源申请量为: 100.0
agent 10 在 9999 次迭代中的资源申请量为: 100.0
agent 11 在 9999 次迭代中的资源申请量为: 100.0
agent 12 在 9999 次迭代中的资源申请量为: 81.93370530741207
agent 13 在 9999 次迭代中的资源申请量为: 100.0
agent 14 在 9999 次迭代中的资源申请量为: 87.4368884395974
agent 15 在 9999 次迭代中的资源申请量为: 89.85381470323325
agent 16 在 9999 次迭代中的资源申请量为: 100.0
agent 17 在 9999 次迭代中的资源申请量为: 100.0
agent 18 在 9999 次迭代中的资源申请量为: 99.40118605819335
agent 19 在 9999 次迭代中的资源申请量为: 96.36580647968138
agent 20 在 9999 次迭代中的资源申请量为: 100.0
agent 21 在 9999 次迭代中的资源申请量为: 100.0
agent 22 在 9999 次迭代中的资源申请量为: 100.0
agent 23 在 9999 次迭代中的资源申请量为: 100.0
agent 24 在 9999 次迭代中的资源申请量为: 85.0561483503012
agent 25 在 9999 次迭代中的资源申请量为: 100.0
agent 26 在 9999 次迭代中的资源申请量

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 100.0
agent 2 在 51999 次迭代中的资源申请量为: 100.0
agent 3 在 51999 次迭代中的资源申请量为: 100.0
agent 4 在 51999 次迭代中的资源申请量为: 82.18608204191318
agent 5 在 51999 次迭代中的资源申请量为: 100.0
agent 6 在 51999 次迭代中的资源申请量为: 100.0
agent 7 在 51999 次迭代中的资源申请量为: 100.0
agent 8 在 51999 次迭代中的资源申请量为: 98.98813646213765
agent 9 在 51999 次迭代中的资源申请量为: 100.0
agent 10 在 51999 次迭代中的资源申请量为: 100.0
agent 11 在 51999 次迭代中的资源申请量为: 100.0
agent 12 在 51999 次迭代中的资源申请量为: 83.60297880427689
agent 13 在 51999 次迭代中的资源申请量为: 100.0
agent 14 在 51999 次迭代中的资源申请量为: 87.9078427731322
agent 15 在 51999 次迭代中的资源申请量为: 91.4093921348826
agent 16 在 51999 次迭代中的资源申请量为: 100.0
agent 17 在 51999 次迭代中的资源申请量为: 100.0
agent 18 在 51999 次迭代中的资源申请量为: 99.68498199996769
agent 19 在 51999 次迭代中的资源申请量为: 96.56868024020838
agent 20 在 51999 次迭代中的资源申请量为: 100.0
agent 21 在 51999 次迭代中的资源申请量为: 100.0
agent 22 在 51999 次迭代中的资源申请量为: 100.0
agent 23 在 51999 次迭代中的资源申请量为: 100.0
agent 24 在 51999 次迭代中的资源申请量为: 84.8431997384078
agent 25 在 51999 次迭代中的资源申请量为: 100.0
a

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 100.0
agent 2 在 53999 次迭代中的资源申请量为: 100.0
agent 3 在 53999 次迭代中的资源申请量为: 100.0
agent 4 在 53999 次迭代中的资源申请量为: 79.39346653333068
agent 5 在 53999 次迭代中的资源申请量为: 100.0
agent 6 在 53999 次迭代中的资源申请量为: 100.0
agent 7 在 53999 次迭代中的资源申请量为: 100.0
agent 8 在 53999 次迭代中的资源申请量为: 98.39386352958405
agent 9 在 53999 次迭代中的资源申请量为: 100.0
agent 10 在 53999 次迭代中的资源申请量为: 100.0
agent 11 在 53999 次迭代中的资源申请量为: 100.0
agent 12 在 53999 次迭代中的资源申请量为: 82.77579238430181
agent 13 在 53999 次迭代中的资源申请量为: 100.0
agent 14 在 53999 次迭代中的资源申请量为: 87.3859942284907
agent 15 在 53999 次迭代中的资源申请量为: 88.57817699207564
agent 16 在 53999 次迭代中的资源申请量为: 100.0
agent 17 在 53999 次迭代中的资源申请量为: 100.0
agent 18 在 53999 次迭代中的资源申请量为: 98.75263207541211
agent 19 在 53999 次迭代中的资源申请量为: 95.80576118342942
agent 20 在 53999 次迭代中的资源申请量为: 100.0
agent 21 在 53999 次迭代中的资源申请量为: 100.0
agent 22 在 53999 次迭代中的资源申请量为: 100.0
agent 23 在 53999 次迭代中的资源申请量为: 100.0
agent 24 在 53999 次迭代中的资源申请量为: 85.76293137860542
agent 25 在 53999 次迭代中的资源申请量为: 100.0

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 100.0
agent 2 在 55999 次迭代中的资源申请量为: 100.0
agent 3 在 55999 次迭代中的资源申请量为: 100.0
agent 4 在 55999 次迭代中的资源申请量为: 82.0792323547214
agent 5 在 55999 次迭代中的资源申请量为: 100.0
agent 6 在 55999 次迭代中的资源申请量为: 100.0
agent 7 在 55999 次迭代中的资源申请量为: 100.0
agent 8 在 55999 次迭代中的资源申请量为: 99.88622284330175
agent 9 在 55999 次迭代中的资源申请量为: 100.0
agent 10 在 55999 次迭代中的资源申请量为: 100.0
agent 11 在 55999 次迭代中的资源申请量为: 100.0
agent 12 在 55999 次迭代中的资源申请量为: 84.47969277686505
agent 13 在 55999 次迭代中的资源申请量为: 100.0
agent 14 在 55999 次迭代中的资源申请量为: 88.95596901480312
agent 15 在 55999 次迭代中的资源申请量为: 91.36391271664375
agent 16 在 55999 次迭代中的资源申请量为: 100.0
agent 17 在 55999 次迭代中的资源申请量为: 100.0
agent 18 在 55999 次迭代中的资源申请量为: 100.0
agent 19 在 55999 次迭代中的资源申请量为: 97.20664539894247
agent 20 在 55999 次迭代中的资源申请量为: 100.0
agent 21 在 55999 次迭代中的资源申请量为: 100.0
agent 22 在 55999 次迭代中的资源申请量为: 100.0
agent 23 在 55999 次迭代中的资源申请量为: 100.0
agent 24 在 55999 次迭代中的资源申请量为: 84.88913936298175
agent 25 在 55999 次迭代中的资源申请量为: 100.0
agent 26 在 

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 100.0
agent 2 在 57999 次迭代中的资源申请量为: 100.0
agent 3 在 57999 次迭代中的资源申请量为: 100.0
agent 4 在 57999 次迭代中的资源申请量为: 81.00595798988212
agent 5 在 57999 次迭代中的资源申请量为: 100.0
agent 6 在 57999 次迭代中的资源申请量为: 100.0
agent 7 在 57999 次迭代中的资源申请量为: 100.0
agent 8 在 57999 次迭代中的资源申请量为: 98.981710755326
agent 9 在 57999 次迭代中的资源申请量为: 100.0
agent 10 在 57999 次迭代中的资源申请量为: 100.0
agent 11 在 57999 次迭代中的资源申请量为: 100.0
agent 12 在 57999 次迭代中的资源申请量为: 82.35876269571745
agent 13 在 57999 次迭代中的资源申请量为: 100.0
agent 14 在 57999 次迭代中的资源申请量为: 87.9766261917046
agent 15 在 57999 次迭代中的资源申请量为: 90.14822320978143
agent 16 在 57999 次迭代中的资源申请量为: 100.0
agent 17 在 57999 次迭代中的资源申请量为: 100.0
agent 18 在 57999 次迭代中的资源申请量为: 99.55363794159318
agent 19 在 57999 次迭代中的资源申请量为: 96.43788577181235
agent 20 在 57999 次迭代中的资源申请量为: 100.0
agent 21 在 57999 次迭代中的资源申请量为: 100.0
agent 22 在 57999 次迭代中的资源申请量为: 100.0
agent 23 在 57999 次迭代中的资源申请量为: 100.0
agent 24 在 57999 次迭代中的资源申请量为: 84.20494295232008
agent 25 在 57999 次迭代中的资源申请量为: 100.0
a

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 100.0
agent 2 在 59999 次迭代中的资源申请量为: 100.0
agent 3 在 59999 次迭代中的资源申请量为: 100.0
agent 4 在 59999 次迭代中的资源申请量为: 80.09008496164323
agent 5 在 59999 次迭代中的资源申请量为: 100.0
agent 6 在 59999 次迭代中的资源申请量为: 100.0
agent 7 在 59999 次迭代中的资源申请量为: 100.0
agent 8 在 59999 次迭代中的资源申请量为: 98.47583472210508
agent 9 在 59999 次迭代中的资源申请量为: 100.0
agent 10 在 59999 次迭代中的资源申请量为: 100.0
agent 11 在 59999 次迭代中的资源申请量为: 100.0
agent 12 在 59999 次迭代中的资源申请量为: 81.86473065159375
agent 13 在 59999 次迭代中的资源申请量为: 100.0
agent 14 在 59999 次迭代中的资源申请量为: 87.25683453626942
agent 15 在 59999 次迭代中的资源申请量为: 89.24470502613745
agent 16 在 59999 次迭代中的资源申请量为: 100.0
agent 17 在 59999 次迭代中的资源申请量为: 100.0
agent 18 在 59999 次迭代中的资源申请量为: 98.86163961587474
agent 19 在 59999 次迭代中的资源申请量为: 96.33069057109898
agent 20 在 59999 次迭代中的资源申请量为: 100.0
agent 21 在 59999 次迭代中的资源申请量为: 100.0
agent 22 在 59999 次迭代中的资源申请量为: 100.0
agent 23 在 59999 次迭代中的资源申请量为: 100.0
agent 24 在 59999 次迭代中的资源申请量为: 83.58333379448914
agent 25 在 59999 次迭代中的资源申请量为: 100.

In [26]:
np.savetxt('./data/00_DGD_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_attack4,fmt='%0.8f')#用户资源
np.savetxt('./data/00_DGD_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_attack4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/00_DGD_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_attack4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/00_DGD_dual_variable_Gaussian_attack_small.txt',dual_variable_attack4,fmt='%0.8f')#用户对偶变量